In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import pickle
import pandas as pd
import numpy as np

# Załaduj wytrenowany model
try:
    with open('model_2.pkl', 'rb') as file:
        model = pickle.load(file)
except Exception as e:
    print("Błąd podczas ładowania modelu:", e)

# Funkcja sigmoidalna
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict_result():
    home_team = combo_home.get()
    away_team = combo_away.get()

    if not home_team or not away_team:
        messagebox.showerror("Błąd", "Proszę wybrać drużyny!")
        return

    # Wczytaj dane
    dataset = pd.read_csv(r'C:\Magisterka\Predicting-the-results-of-Premier-League-matches\TEST\final_dataset_2.csv')

    # Sprawdzenie, czy drużyny istnieją w danych
    teams = dataset['HomeTeam'].unique()
    if home_team not in teams or away_team not in teams:
        messagebox.showerror("Błąd", "Jedna z drużyn nie istnieje w zbiorze danych.")
        return

    # Pobranie danych drużyn
    home_team_data = dataset[dataset['HomeTeam'] == home_team].iloc[0]
    away_team_data = dataset[dataset['AwayTeam'] == away_team].iloc[0]

    # Przygotowanie danych wejściowych do predykcji (bez duplikowania cech)
    features = ['HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HTFormPts', 'ATFormPts']
    input_data = pd.DataFrame([home_team_data[features].values.tolist() + away_team_data[features].values.tolist()],
                              columns=features + features)

    # Usunięcie duplikujących się cech
    input_data = input_data.loc[:, ~input_data.columns.duplicated()]

    # Sprawdzenie, jakie cechy modelu zostały użyte podczas trenowania
    model_features = model.feature_names_in_

    # Wyświetlenie cech modelu i cech danych wejściowych
    print(f"Cecha modelu: {model_features}")
    print(f"Cecha wejściowa: {input_data.columns}")

    # Dopasowanie kolumn do tych, które były używane w trakcie trenowania
    input_data = input_data[model_features]

    # Predykcja
    try:
        prediction = model.predict(input_data)
        
        # Zastosowanie funkcji sigmoidalnej, aby uzyskać prawdopodobieństwo
        probability = sigmoid(prediction[0])  # Przekształcenie wyniku regresji na prawdopodobieństwo

        # Obliczenie procentowego wyniku
        result_message = f"Szansa na wygraną {home_team}: {probability*100:.2f}%"
        messagebox.showinfo("Wynik analizy", result_message)
    
    except Exception as e:
        messagebox.showerror("Błąd", f"Wystąpił błąd: {str(e)}")

# Utworzenie głównego okna aplikacji
root = tk.Tk()
root.title("Przewidywanie wyniku meczu")
root.geometry("600x350")  # Zwiększenie rozmiaru okna

# Dodanie etykiety
label = tk.Label(root, text="Wprowadź drużyny", font=("Arial", 16, 'bold'))
label.grid(row=0, column=0, columnspan=2, pady=20)

# Etykieta i pole rozwijane dla drużyny gospodarzy
label_home = tk.Label(root, text="Drużyna gospodarzy", font=("Arial", 12))
label_home.grid(row=1, column=0, pady=10, sticky="w", padx=20)
combo_home = ttk.Combobox(root, font=("Arial", 12), width=30)
combo_home.grid(row=1, column=1, pady=10, padx=20)

# Etykieta i pole rozwijane dla drużyny gości
label_away = tk.Label(root, text="Drużyna gości", font=("Arial", 12))
label_away.grid(row=2, column=0, pady=10, sticky="w", padx=20)
combo_away = ttk.Combobox(root, font=("Arial", 12), width=30)
combo_away.grid(row=2, column=1, pady=10, padx=20)

# Wczytaj drużyny z pliku do list rozwijanych
dataset = pd.read_csv(r'C:\Magisterka\Predicting-the-results-of-Premier-League-matches\TEST\final_dataset_2.csv')
teams = dataset['HomeTeam'].unique()
combo_home['values'] = sorted(teams)
combo_away['values'] = sorted(teams)

# Dodanie przycisku do wykonania analizy
button_predict = tk.Button(root, text="Przewiduj wynik", font=("Arial", 14, 'bold'), command=predict_result)
button_predict.grid(row=3, column=0, columnspan=2, pady=20)

# Uruchomienie aplikacji
root.mainloop()


Cecha modelu: ['HTGS' 'ATGS' 'HTGC' 'ATGC' 'HTP' 'ATP' 'HTFormPts' 'ATFormPts']
Cecha wejściowa: Index(['HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HTFormPts', 'ATFormPts'], dtype='object')
Cecha modelu: ['HTGS' 'ATGS' 'HTGC' 'ATGC' 'HTP' 'ATP' 'HTFormPts' 'ATFormPts']
Cecha wejściowa: Index(['HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HTFormPts', 'ATFormPts'], dtype='object')
